1. What are the advantages of a CNN over a fully connected DNN for image classification?

CNNs have several advantages over a fully connected DNN for certain kinds of problems:

- They use significantly fewer weights. Kernels are typically small, like 1x1, 3x3, 5x5, or 7x7, and these kernels are applied across an entire channel. This allows the model to scale to very large input image sizes without having an explosion in the number of weights as a function of depth. The reduced number of weights in combination with downsampling via pooling allow CNNs to increase the number of channels (feature maps) at higher layers, given the model the ability to learn a diverse array of patterns.
- Invariances:
  - translation invariance
  - slight rotation invariance
- By focusing on small patches in input channels, CNNs can discover patterns that happen to be prevalent in real world images. These primitive patterns like lines, circles, corners, etc. can be built upon in successive layers, building up to high level features that can be localized to patches in higher level layers.
- Weight sharing is a form of regularization

2. Consider a CNN composed of three convolutional layers, each with 3 × 3 kernels, a stride of 2, and "same" padding. The lowest layer outputs 100 feature maps, the middle one outputs 200, and the top one outputs 400. The input images are RGB images of 200 × 300 pixels.
What is the total number of parameters in the CNN? If we are using 32-bit floats, at least how much RAM will this network require when making a prediction for a single instance? What about when training on a mini-batch of 50 images?


weights:
- 100 feature maps * ((3w * 3h * 3d) kernel weights + 1 bias weight) = 2,800
- 200 feature maps * ((3w * 3h * 100d) kernel weights + 1 bias weight) = 180,200
- 400 feature maps * ((3w * 3h * 200d) kernel weights + 1 bias weight) = 720,400

Since no fully connected layer is mentioned, the total number of weights will be 903,400.

4 bytes/weight * 903,400 weights = 3,613,300 bytes

memory usage:
- 4 bytes/pixel * (200 * 300 pixels * 3 channels) = 720,000 bytes
- 4 bytes/pixel * (100 * 150 pixels * 100 feature maps) = 6,000,000 bytes
- 4 bytes/pixel * (50 * 75 pixels * 200 feature maps) = 3,000,000 bytes
- 4 bytes/pixel * (25 * 38 pixels * 400 feature maps) = 1,520,000 bytes

During inference, the previous layer's inputs can be dropped from memory after they are used. If we assume that we keep the entire model in memory, the maximum memory usage would be:
9,000,000 + 3,613,300 bytes = 12,613,300 bytes ~= 12.03 MiB

The model memory usage will not increase for batches, while the input memory will. We must keep all computations in memory for backpropagation, so for a batch of 50 we would have:

size of all computations:
720,000 + 6,000,000 + 3,000,000 + 1,520,000 = 11,240,000 bytes

(50 inputs * 11,520,000) + 3,613,300 = 579613300 bytes ~= 552.76 MiB

3. If your GPU runs out of memory while training a CNN, what are five things you could try to solve the problem?

- Reduce the kernel sizes
- Reduce the number of feature maps per layer
- Use more max/avg pooling
- Add a stride > 1
- use float16 instead of float32
- Reduce batch size
- Buy a GPU with more memory
- Model parallel training

4. Why would you want to add a max pooling layer rather than a convolutional layer with the same stride?

If the purpose is downsampling, max pooling does not have any parameters, and so costs less in terms of model size.

5. When would you want to add a local response normalization layer?

Local response normalization encourages only a single neuron in a depthwise slice to have a strong activation. This encourages diversity across feature maps and improves generalization. GoogLeNet uses LRN relatively early on, as the 3rd and 6th hiden layers. This gives the higher layers more feature diversity for pattern discovery.

6. Can you name the main innovations in AlexNet, compared to LeNet-5? What about the main innovations in GoogLeNet, ResNet, SENet, and Xception?

AlexNet
- Much larger and deeper than LeNet-5
- Stacked convolutional layers directly together instead of always using pooling between
- Used Local Response Normalization
- Dropout in highest layers
- Data augmentation (shift, flip, and alter lighting)
- 10 hidden layers

GoogLeNet:
- Considerably deeper than AlexNet (20 hidden layers)
- Inception modules:
  - Depthwise concatenation of different convolutions of inputs
  - Including stacked convolutions with 1x1 kernel "bottleneck" layers that can still find depthwise patterns
- Global average pooling
  - reduces number of params instead of fully connected layer
  - reduces risk of overfitting
  
ResNet:
- 152 layers
- residual layers
  - skip connections allow gradients to propogate to lower layers even if some layers block the gradient
  - often the target function is close to the identity function, which is what a residual layer produces with weights randomly initialized around 0
- uses batch normalization
- no pooling between residual layers in middle portions of the network
  - uses stride for downsampling

Xception:
- merged ideas for ResNet and GoogLeNet
- while not novel, more heavy use of depthwise separable convolution instead of inception modules

SENet:
- extends ResNets and inception networks
- Adds Squeeze-and-Excitation blocks (SE blocks) to inception modules and residual units
- SE block attenuates small (irrelevant) outputs, while passing large outputs through across the depth dimension
  - global average pooling across each feature map
  - squeeze layer at significantly lower dimensionality forces learning a general representation of feature combinations
  - sigmoid output layer normalizes output to range (0, 1)

7. What is a fully convolutional network? How can you convert a dense layer into a convolutional layer?

A fully convolutional network makes the observation that the final fully connected layer is doing the same computations as a convolutional layer with a kernel size the same as the size of the feature maps in the previous layers. Each neuron in a fully connected layer is connected to each cell in every feature map. In a convolutional layer with kernel size matching the width and height of each feature map, the same is true. The kernel is connected to every pixel in every feature map. This observation makes it possible to process inputs of variable size by using convolution instead of the final fully connected layer. There will be a minimum size for the input, but inputs larger than that will just result in feature maps larger than 1x1 in the output convolutional layer. The depth dimension represents the same thing as the output neurons in the fully connected layer. This is an extremely efficient way to make predictions across an input by dividing it up into cells and making predictions for each cell.

8. What is the main technical difficulty of semantic segmentation?

CNNs lose spatial resolution in higher layers due to strides greater than 1. Since the task in semantic segmentation is to assign a class to every pixel, this makes it challenging to build an efficient network that can still have access to enough spatial resolution to make accurate predictions per pixel.

One solution is an upsampling layer. One approach to upsampling is a transposed convolutional layer. This approach inserts empty rows and columns into the target feature map (the number of empty rows and columns is specified by stride), and then fills it in by doing regular convolution at stride 1 over the source feature maps.

In order to regain lost spatial resolution, upsampling can be performed in steps, increasing the spatial resolution from higher level feature maps, by adding in the output of lower level feature maps with matching feature map sizes.